# Histogram Model for Image Objects with Depth
----
Sergei Papulin (papulin.edu@gmail.com)

## Contents

- [Loading Dataset](#Loading-Dataset)
- [Defining Positional Elements](#Defining-Positional-Elements)
- [Defining Object Elements](#Defining-Object-Elements)
- [Defining Depth Elements](#Defining-Depth-Elements)
- [Creating Histogram](#Creating-Histogram)
- [Querying](#Querying)
- [Image Retrieval](#Image-Retrieval)
- [References](#References)

### Creating virtual environment

⚠️ **Warning.** You need at least 8GB RAM available to load the dataset

This is an optional step. You can skip it and install packages to your current environment.

```bash
python -m venv .venv/histtest
source .venv/histtest/bin/activate
pip install \
    numpy==1.19.5 \
    matplotlib==3.0.3 \
    jupyter==1.0.0 \
    pillow==5.4.1 \
    scikit-image==0.14.2 \
    mat73==0.55 \
    himpy=0.0.1
```

#### Load packages

In [ ]:
# import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from himpy.histogram import operations
from himpy.executor import Parser, Evaluator
from himpy.utils import E

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../")

from utils.datasets import NYULoader

# feature extraction
from utils.feature_extraction import (
    FeatureMerger,
    PositionSetTransformer,
    filter_data,
    create_histogram,
    create_histogram_,
    extract_elements,
    extract_element_set
)

from utils.feature_extraction.nyuv2 import (
    NYUObjectSetTransformer,
    NYUDepthSetTransformer,
    load_nyu_histograms
)

# search engine
from utils.search_engine import SearchEngine

# plot
from utils.plot.matplotlib_plot import (
    plot_position_grid,
    plot_object_ids,
    show_operation_result,
    show_retrieved_images
)

## Loading Dataset


Download the dataset from the NYU Depth Dataset V2 [website](https://cs.nyu.edu/~silberman/datasets/nyu_depth_v2.html):
- [labeled dataset](http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat)

**Basic dataset fileds**

| Key | Description | 
|:--:|:--|
| depths | HxWxN matrix of in-painted depth maps where H and W are the height and width, respectively and N is the number of images. The values of the depth elements are in meters. |
| images | HxWx3xN matrix of RGB images where H and W are the height and width, respectively, and N is the number of images. |
| instances | HxWxN matrix of instance maps. Use get_instance_masks.m in the Toolbox to recover masks for each object instance in a scene. |
| labels | HxWxN matrix of object label masks where H and W are the height and width, respectively and N is the number of images. The labels range from 1..C where C is the total number of classes. If a pixel’s label value is 0, then that pixel is ‘unlabeled’. |
| names | Cx1 cell array of the english names of each class. |
| namesToIds | map from english label names to class IDs (with C key-value pairs) |
| scenes | Nx1 cell array of the name of the scene from which each image was taken. |
| sceneTypes | Nx1 cell array of the scene type from which each image was taken. |

### Downloading Dataset

In [ ]:
loader = NYULoader()

In [ ]:
data = loader.fetch_load()

In [ ]:
data.keys()

### Images

In [ ]:
# Id of some image from the dataset
IMAGE_INDX = 15

In [ ]:
# Plot the image
I = data["images"][IMAGE_INDX]
plt.imshow(I)
plt.title("Image")
plt.show()

## Defining Positional Elements

### Low-Level Elements

In [ ]:
# Grid params: 5 splits along Y, and 5 along X
GRID = (5, 5)

# Create a position transformer
position_transformer = PositionSetTransformer(splits=GRID, element_ndim=3)

# Set an image size
# Note: Actually there is no fitting here, just follow the common interface
position_transformer.fit(X=I, y=None)

# Build an image in which each pixel defines a position
position_image = position_transformer.transform(X=I, batch=False)

In [ ]:
# Plot the positional element along with the initial image
fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].axis("off")
axes[1].set_title("Low-level position elements")
axes[1].imshow(position_image)
axes[1] = plot_position_grid(position_transformer, axes[1])
axes[1].axis("off")
axes[2].set_title("Matching image and elements")
axes[2].imshow(I)
axes[2] = plot_position_grid(position_transformer, axes[2])
axes[2].axis("off")
plt.show()

### High-Level Elements

In [ ]:
# Create an instance of query parser
parser = Parser()

In [ ]:
# Definition of high-level positional elements

Ep_top    = E("1+2+3+4+5+6+7+8+9+10")
Ep_bottom = E("16+17+18+19+20+21+22+23+24+25")
Ep_left   = E("1+2+6+7+11+12+16+17+21+22")
Ep_right  = E("4+5+9+10+14+15+19+20+24+25")
Ep_center = E("7+8+9+12+13+14+17+18+19")

Eps = [
    ("top", Ep_top), 
    ("bottom", Ep_bottom), 
    ("left", Ep_left), 
    ("right", Ep_right), 
    ("center", Ep_center)
]


# Sets of high-level positional elements (they will be used for the Evaluator below)

Eps_set = { name: parser.parse_set(Ep.value) for name, Ep in Eps}
Eps_set["center"]

In [ ]:
# Plot a high-level element
Ep_set_ = Eps_set["center"]

elements_image = position_transformer.filter_elements(position_image, Ep_set_)
filtered_image = position_transformer.filter_data(position_image, I, Ep_set_)

fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].axis("off")
axes[1].set_title("High-level position elements")
axes[1].imshow(elements_image)
axes[1] = plot_position_grid(position_transformer, axes[1], Ep_set_)
axes[1].axis("off")
axes[2].set_title("Matching image and elements")
axes[2].imshow(filtered_image)
axes[2] = plot_position_grid(position_transformer, axes[2], Ep_set_)
axes[2].get_xaxis().set_visible(False)
axes[2].get_yaxis().set_visible(False)

plt.show()

## Defining Object Elements

### Low-Level Elements

In [ ]:
print("Total number of classes: {}\n".format(len(data["names"])))
print("Classes: {}\n".format(", ".join(cl[0] for cl in data["names"])))

In [ ]:
# Create object transformer
object_transformer = NYUObjectSetTransformer(data)

# Create a mask of objects
object_image = object_transformer.fit_transform(X=None, y=None, ids=IMAGE_INDX)

# Mask image based on segments
image__object_filtered = object_transformer.filter_data(object_image, I)

In [ ]:
# Plot initial image, object image, and filtered image
fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].axis("off")
axes[1].set_title("Low-level object elements")
axes[1].imshow(object_image)
axes[1] = plot_object_ids(object_image, axes[1])
axes[1].axis("off")
axes[2].set_title("Matching image and elements")
axes[2].imshow(image__object_filtered)
axes[2] = plot_object_ids(object_image, axes[2])
axes[2].axis("off")

plt.show()

In [ ]:
# Names of image objects
object_transformer.get_names(extract_elements(object_image))

### High-Level Elements

In [ ]:
# Definition of high-level positional elements

Eo_room    = E("4+11+21")
Eo_table   = E("19")
Eo_printer = E("66")
Eo_ceiling = E("4")
Eo_door    = E("28")

Eos = [
    ("room", Eo_room),
    ("table", Eo_table),
    ("printer", Eo_printer),
    ("ceiling", Eo_ceiling),
    ("door", Eo_door),
]


# Sets of high-level positional elements (they will be used for the Evaluator below)

Eos_set = { name: parser.parse_set(Eo.value) for name, Eo in Eos}
Eos_set["room"]

In [ ]:
# Plot a high-level element
Eo_set_ = Eos_set["room"]

elements_image = object_transformer.filter_elements(object_image, element_ids=Eo_set_)
filtered_image = object_transformer.filter_data(object_image, I, element_ids=Eo_set_)

fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].axis("off")
axes[1].set_title("High-level object elements")
axes[1].imshow(elements_image)
axes[1] = plot_object_ids(object_image, axes[1], Eo_set_)
axes[1].axis("off")
axes[2].set_title("Matching image and elements")
axes[2].imshow(filtered_image)
axes[2] = plot_object_ids(object_image, axes[2], Eo_set_)
axes[2].get_xaxis().set_visible(False)
axes[2].get_yaxis().set_visible(False)
plt.show()

In [ ]:
# Names of image objects
object_transformer.get_names(Eo_set_, return_id=True)

## Defining Depth Elements

### Low-Level Elements

In [ ]:
# Create object transformer
depth_transformer = NYUDepthSetTransformer(data, nbins=10)

# Create a mask of objects
depth_image = depth_transformer.fit_transform(X=None, y=None, ids=IMAGE_INDX)

# Mask image based on segments
image__depth_filtered = depth_transformer.filter_data(depth_image, I)

In [ ]:
# Plot initial image, object image, and filtered image
fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].axis("off")
axes[1].set_title("Low-level object elements")
axes[1].imshow(depth_image)
axes[1] = plot_object_ids(depth_image, axes[1])
axes[1].axis("off")
axes[2].set_title("Matching image and elements")
axes[2].imshow(image__depth_filtered)
axes[2].imshow(depth_image, alpha=0.4)
axes[2] = plot_object_ids(depth_image, axes[2])
axes[2].axis("off")

plt.show()

### High-Level Elements

In [ ]:
# Definition of high-level positional elements

Ed_faraway = E("7+8+9+10")
Ed_notfar  = E("3+4+5+6")
Ed_near    = E("1+2")


Eds = [
    ("faraway", Ed_faraway),
    ("notfar", Ed_notfar),
    ("near", Ed_near),
]


# Sets of high-level positional elements (they will be used for the Evaluator below)

Eds_set = { name: parser.parse_set(Ed.value) for name, Ed in Eds}
Eds_set["faraway"]

In [ ]:
# Plot a high-level element
Ed_set_ = Eds_set["faraway"]

elements_image = depth_transformer.filter_elements(depth_image, element_ids=Ed_set_)
filtered_image = depth_transformer.filter_data(depth_image, I, element_ids=Ed_set_)

fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].axis("off")
axes[1].set_title("High-level object elements")
axes[1].imshow(elements_image)
axes[1] = plot_object_ids(depth_image, axes[1], Ed_set_)
axes[1].axis("off")
axes[2].set_title("Matching image and elements")
axes[2].imshow(filtered_image)
axes[2] = plot_object_ids(elements_image, axes[2], Ed_set_)
axes[2].get_xaxis().set_visible(False)
axes[2].get_yaxis().set_visible(False)
plt.show()

## Creating Histogram

In [ ]:
# Option 1
hist = create_histogram((position_image, depth_image, object_image))
hist.to_dict()

In [ ]:
# Option 2.a Merge features into a single image
feature_merger = FeatureMerger()
merged_image = feature_merger.fit_transform((position_image, depth_image, object_image))
merged_image

In [ ]:
# Option 2.b Create a histogram
hist = create_histogram_(merged_image)
hist.to_dict()

## Querying

In [ ]:
high_level_elements = {
    0: Eps_set, # positions
    1: Eds_set, # depths
    2: Eos_set  # objects
}

In [ ]:
evaluator = Evaluator(operations, hist, high_level_elements=high_level_elements)

In [ ]:
E1 = E("left", "faraway", "room")
E2 = E("center", "near", "printer")

In [ ]:
E1_expr = parser.parse_string(E1.value)
HE1 = evaluator.eval(E1_expr)
print("Expression for E1:\n{}".format(E1.value))
print("\nThe parsed expressino for E1 in the postfix notation:\n{}".format(E1_expr))
print("\nHistogram of E1 given the image:\n{}".format(HE1.to_dict()))
print("\nValue of presence for E1:\n{}".format(HE1.sum()))

In [ ]:
E2_expr = parser.parse_string(E2.value)
HE2 = evaluator.eval(E2_expr)
print("Expression for E2:\n{}".format(E2.value))
print("\nThe parsed expressino for E2 in the postfix notation:\n{}".format(E2_expr))
print("\nHistogram of E2 given the image:\n{}".format(HE2.to_dict()))
print("\nValue of presence for E2:\n{}".format(HE2.sum()))

In [ ]:
# Plot histogram elements

E1_set = extract_element_set(HE1, 2)
E2_set = extract_element_set(HE2, 2)

E1_image = filter_data(I, merged_image, HE1.elements())
E2_image = filter_data(I, merged_image, HE2.elements())

fig, axes = plt.subplots(1, 3, figsize=(14,20))
axes[0].set_title("Initial Image")
axes[0].imshow(I)
axes[0].get_xaxis().set_visible(False)
axes[0].get_yaxis().set_visible(False)
axes[1].set_title("E1: {}".format(E1.value))
axes[1].imshow(I)
axes[1].imshow(E1_image, alpha=0.8)
axes[1] = plot_position_grid(position_transformer, axes[1], E1_set[0])
axes[1].get_xaxis().set_visible(False)
axes[1].get_yaxis().set_visible(False)
axes[2].set_title("E2: {}".format(E2.value))
axes[2].imshow(I)
axes[2].imshow(E2_image, alpha=0.8)
axes[2] = plot_position_grid(position_transformer, axes[2], E2_set[0])
axes[2].get_xaxis().set_visible(False)
axes[2].get_yaxis().set_visible(False)

plt.show()

### Operations on Histogram Elements

#### Example for Union

In [ ]:
# Expression with union
E_union = E1 + E2

# Parsed expression
E_union_expr = parser.parse_string(E_union.value)

# Calculate histogram value
HE_union = evaluator.eval(E_union_expr)

print("Expression for E_union:\n{}".format(E_union))
print("\nThe parsed expression for E_union in the postfix notation:\n{}".format(E_union_expr))
print("\nHistogram of E_union given the image:\n{}".format(HE_union.to_dict()))
print("\nValue of presence for E_union:\n{}".format(HE_union.sum()))

In [ ]:
# Extract ids of non-zero elements for each feature
E_result_set = extract_element_set(HE_union, 2)

In [ ]:
# Plot elements and result
transformers = (position_transformer, depth_transformer, object_transformer)
titles = ["E1: {}".format(E1), "E2: {}".format(E2), "Result: {}".format(E_union)]
show_operation_result(I, merged_image, "f2", HE1, HE2, HE_union, transformers, titles)

#### Other operations

In [ ]:
operation_list = [
    # set operations
    ("union",          "+",    E1 + E2), 
    ("intersection",   "*",    E1 * E2),
    ("substraction",   "-",    E1 - E2),  # or exception, or E1.Sub(E2)
    # logic operations
    ("and",            "&",    E1 & E2),  # or E1.And(E2)
    ("or",             "|",    E1 | E2),  # or E1.Or(E2)
    ("xor",            "^",    E1 ^ E2),  # or E1.Xor(E2)
    ("xsubstraction",  "Xsub", E1.Xsub(E2)),
]

In [ ]:
for op_name, op_sign, op in operation_list:
    E_expr = parser.parse_string(op.value)
    HE = evaluator.eval(E_expr)
    print("{:12}{:^12}{:10}".format("Operation", "Sign", "Result"))
    print("{}".format("-"*34))
    print("{:12}{:^12}{:.5f}".format(op_name, op_sign, HE.sum()))
    E_result_set = extract_element_set(HE, 2)
    titles = ["E1: {}".format(E1), "E2: {}".format(E2), "Result: {}".format(op)]
    show_operation_result(I, merged_image, "f2", HE1, HE2, HE, transformers, titles)

## Image Retrieval

### Expression as query

In [ ]:
# Load histograms of images if exist, otherwise transform images to histograms and return
hists = load_nyu_histograms(data)

In [ ]:
# Initialize a search engine
search_engine = SearchEngine(hists, parser, evaluator)

In [ ]:
TOP_N = 20

In [ ]:
# Define your query
query = E("center", "faraway", "door") & E("right", "notfar", "table")

# Retrieve images using the query
ranked_images = search_engine.retrieve(query, topN=TOP_N)
print("Total retrieved images:", len(ranked_images))
ranked_images[:5]

In [ ]:
# Show top ranked images
show_retrieved_images(
    ranked_images, 
    images=data["images"], 
    limit=TOP_N, 
    title="Query: {}".format(query.value)
)

### Sample image as query

In [ ]:
# Take a sample image and its histogram from the list of histograms that were created previously
sample_image_id = hists[4][0]
sample_hist = hists[4][1]

In [ ]:
# Show the sample image
I = data["images"][sample_image_id]
plt.imshow(I)
plt.title("Image")
plt.show()

In [ ]:
# Retrieve images similar to the sample
ranked_images__sample = search_engine.retrieve(sample_hist, topN=TOP_N)
print("Total retrieved images:", len(ranked_images__sample))
ranked_images__sample[:5]

In [ ]:
# Show top ranked images
show_retrieved_images(
    ranked_images__sample, 
    images=data["images"],
    limit=TOP_N,
    title="Query: {}".format("Sample Image")
)

## References

- [NYU Depth Dataset V2](https://cs.nyu.edu/~silberman/datasets/nyu_depth_v2.html)
- Papulin S. [Introduction to Histogram Model](https://htmlpreview.github.io/?https://github.com/LSHist/histogram/blob/master/docs/hm_basics.html)
- Papulin S. [Multidimensional Histogram Model](https://htmlpreview.github.io/?https://github.com/LSHist/histogram/blob/master/docs/hm_multidim.html)